In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import flopy
import geopandas as gpd
import shapely

### Loading and modifying a MODFLOW 6 model

A existing MODFLOW 6 model will be loaded and modified to include a river discretized using a shapefile..

The model we are using is the model developed by Freyberg () The model domain is into 1 layer, 40 rows, and 20 columns. 

Before loading the simulation object you should define the simulation workspace (`sim_ws`) where the model files are, the simulation name (`name`), and the name and path of the model executable (`exe_name`).  The `sim_ws` should be set to `'data/ex01b'`, `name` should be set to `ex01b`, and `exe_name` should be set to `bin/mf6`.

In [ ]:
ws = 'work/ex03'
name = 'ex03'
exe_name = 'mf6'
load_ws = 'data/freyberg_mf6/'

Load a simulation object using `flopy.mf6.MFSimulation().load()`.

Change the simulation workspace using `sim.set_sim_path()` so that we don't overwrite the original datasets.

Retrieve the gwf model object using `sim.get_model()`. First use `sim.model_names` to get the name of the groundwater model.

#### Create the river package

Load the river shapefile using geopandas. 

In [ ]:
spth = 'data/freyberg/GIS/Flowline_river.shp'
rshp = gpd.read_file(spth)

Plot the shapefile on the model grid.

In [ ]:
ax = rshp.plot()
gwf.modelgrid.plot(ax=ax)

Determine what is in the geometry with `rshp.geometry`.

Create a intersection object from the modelgrid

In [ ]:
ixobj = flopy.utils.gridintersect.GridIntersect(gwf.modelgrid, method='structured')

Intersect the geometry object and the river shapefile goemetry

In [ ]:
rint = ixobj.intersect_linestring(rshp.geometry[0])

Plot the intersected river and the model grid

In [ ]:
ax = ixobj.plot_linestring(rint)
gwf.modelgrid.plot(ax=ax)

What is in the intersected river object? It is a recarray so lets output the dtype.names.

The zero-based row and column locations are in the `'cellids'` column. 

The lengths of each river component are in the `'lengths'` column. Set a `rlen` variable to `rint['lengths']`.

The river is located in layer 1 and the zero-based row, column locations in `rint['cellids']`. The river stage stage and bottom are at 20.1 and 20.0 in row 1, respectively. The river stage stage and bottom are at 11.25 and 10.25 in row 40, respectively. The river conductance is 0.5e-1.

We are going to linearly interpolate the river stage and bottom from the values in row 1 and row 40. For that we will need the total length of the river.

In [ ]:
rlentot = rint['lengths'].sum()
rlentot

The slope of the river stage and river bottom are $\frac{11.25 - 20.1}{rlentot}$ and $\frac{10.25 - 20.0}{rlentot}$, respectively. 

In [ ]:
rstage_us, rbot_us = 20.1, 20.0
rstage_ds, rbot_ds = 11.25, 10.25

In [ ]:
dstage, drbot = (rstage_ds - rstage_us) / rlentot, (rbot_ds - rbot_us) / rlentot
dstage, drbot

Create numpy arrays for the interpolated river stage and river bottom that will be used to create the river package.

In [ ]:
rstage = np.zeros(rint.shape[0], dtype=np.float)
rbot = np.zeros(rint.shape[0], dtype=np.float)

Calculate interpolated river stage and river bottom.

In [ ]:
rlencum = 0.
for idx in range(rstage.shape[0]):
    if idx == 0:
        rlencum = 0.5 * rlen[idx]
    else:
        rlencum += 0.5 * (rlen[idx-1] + rlen[idx])
    rstage[idx] = rstage_us + rlencum * dstage
    rbot[idx] = rbot_us + rlencum * drbot

In [ ]:
rstage

In [ ]:
rbot

Remove the existing river package. Use `gwf.package_names` to determine the name of the river package.

Remove the existing river package using `gwf.remove_package()`.

Create a list of lists with the river package data. The river package data has the form:

    [[(k, i, j), rstage, conductance, rbot, iface],
                       .
                       .
                       .
     [(k, i, j), rstage, conductance, rbot, iface]]

`iface` is an auxillary variable that is used by MODPATH. We will assume that the river is on the top of the model cell (`IFACE` = 6).

Create the new river package using `flopy.mf6.ModflowGwfriv()`.

#### Write the model files and run the model

In [ ]:
sim.write_simulation()
sim.run_simulation()

#### Post-process the results

Load the heads and face flows from the hds and cbc files. The head file can be loaded with the `flopy.utils.HeadFile()` method. The cell-by-cell file can be loaded with the `flopy.utils.CellBudgetFile()` method. MODFLOW 6 creates double precision binary files so `precision='double'` needs to be passed to the `flopy.utils.CellBudgetFile()` method. The `flopy.utils.HeadFile()` method autodetects the precision of the head file. The precision of the head file is a attribute on the head file object (`hobs.precision`) and could be passed as the `precision` parameter.

In [ ]:
hname = gwf.oc.head_filerecord.array[0][0]
cname = gwf.oc.budget_filerecord.array[0][0]
hname, cname

In [ ]:
fpth = os.path.join(ws, hname)
hobj = flopy.utils.HeadFile(fpth)

In [ ]:
hds = hobj.get_data()

In [ ]:
fpth = os.path.join(ws, cname)
cobj = flopy.utils.CellBudgetFile(fpth, precision=hobj.precision)

The entries in the cell-by-cell file can be determined with the `.list_unique_records()` method on the cell budget file object.

In [ ]:
cobj.list_unique_records()

Retrieve the `'DATA-SPDIS'` data type from the cell-by-cell file.

In [ ]:
spd = cobj.get_data(text='DATA-SPDIS')[0]

#### Plot the results

Plot the results using `flopy.plot.PlotMapView()`. The head results can be plotted using the `.plot_array()` method. The discharge results can be plotted using the `plot_specific_discharge()` method. Boundary conditions can be plotted using the `.plot_bc()` method.

In [ ]:
mm = flopy.plot.PlotMapView(model=gwf, layer=0)
mm.plot_array(hds, masked_values=[1e30])

fig = plt.gcf()
fig.set_size_inches(3, 6)

mm.plot_bc('RIV')
mm.plot_bc('WEL')
mm.plot_specific_discharge(spd)
